In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow import nn
import pandas as pd
import numpy as np
import pickle

In [2]:
with open('EDA_x_train.pkl', 'rb') as f:
    x_train = pickle.load(f)

with open('EDA_y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

with open('EDA_x_valid.pkl', 'rb') as f:
    x_valid = pickle.load(f)

with open('EDA_y_valid.pkl', 'rb') as f:
    y_valid = pickle.load(f)

with open('EDA_x_test.pkl', 'rb') as f:
    x_test = pickle.load(f)

with open('EDA_y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

In [3]:
print(x_train.shape, x_valid.shape, y_train.shape, y_valid.shape)
print(x_test.shape,y_valid.shape, y_test.shape)

(24500,) (1121,) (24500,) (1121,)
(2567,) (1121,) (2567,)


In [4]:
class TransformerBlock(layers.Layer):  #공통적인 트랜스포머 블럭
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = MultiHeadAttention(embedding_dim=embed_dim, num_heads=num_heads)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.dropout3 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        #print('어텐션 후',attn_output.shape)
        attn_output = self.dropout1(attn_output, training=training)
        #print('드롭아웃',attn_output.shape)
        out1 = self.layernorm1(inputs + attn_output)
        #print('레이어놈',out1.shape)
        ffn_output = self.ffn(out1)
        #print('ffn후',ffn_output.shape)
        ffn_output = self.dropout2(ffn_output, training=training)
        print("트랜스포머")
        return self.layernorm2(out1 + ffn_output)
class MultiHeadAttention(tf.keras.layers.Layer): #공통적인 트랜스포머 멀티헤드어텐션
    def __init__(self, embedding_dim, num_heads=4):
        super(MultiHeadAttention, self).__init__()
        self.embedding_dim = embedding_dim # d_model
        self.num_heads = num_heads

        assert embedding_dim % self.num_heads == 0

        self.projection_dim = embedding_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embedding_dim)
        self.key_dense = tf.keras.layers.Dense(embedding_dim)
        self.value_dense = tf.keras.layers.Dense(embedding_dim)
        self.dense = tf.keras.layers.Dense(embedding_dim)

    def scaled_dot_product_attention(self, query, key, value):
        matmul_qk = tf.matmul(query, key, transpose_b=True)
        depth = tf.cast(tf.shape(key)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)
        attention_weights = tf.nn.softmax(logits, axis=-1)
        output = tf.matmul(attention_weights, value)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]

        # (batch_size, seq_len, embedding_dim)
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)

        # (batch_size, num_heads, seq_len, projection_dim)
        query = self.split_heads(query, batch_size)  
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        scaled_attention, _ = self.scaled_dot_product_attention(query, key, value)
        # (batch_size, seq_len, num_heads, projection_dim)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  

        # (batch_size, seq_len, embedding_dim)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim))
        outputs = self.dense(concat_attention)
        return outputs

In [5]:
from tensorflow.keras import backend as K
def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    # return a single tensor value
    return recall


def precision(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Positive) = 예측 값이 1(Positive) 전체
    count_true_positive_false_positive = K.sum(y_pred_yn)

    # Precision = (True Positive) / (True Positive + False Positive)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())

    # return a single tensor value
    return precision


def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    
    # return a single tensor value
    return _f1score


In [6]:
class eda_TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, max_len, embed_dimm):
        super(eda_TokenAndPositionEmbedding, self).__init__()
        self.temp_emb = tf.keras.layers.Embedding(input_dim=max_len, output_dim=embed_dim)
        self.pos_emb = tf.keras.layers.Embedding(input_dim=max_len, output_dim=embed_dim)

    def call(self, x):
        maxlen = 80
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.temp_emb(x)
        return x + positions

In [7]:
embed_dim = 32 # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen=50
training=1

eda_input = layers.Input(shape=(80,)) #
eda_embedding_layer = eda_TokenAndPositionEmbedding(80, embed_dim)
eda_embedding = eda_embedding_layer(eda_input)  #

transformer_block = TransformerBlock(32, num_heads, ff_dim) 

output = transformer_block(eda_embedding)
output = transformer_block(output)

x = layers.GlobalAveragePooling1D()(output)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="sigmoid")(x)
x = layers.Dropout(0.1)(x)

outputs = layers.Dense(7, activation="softmax")(x)  
print(outputs.shape)

model = keras.Model(inputs=eda_input, outputs=outputs)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

트랜스포머
트랜스포머
(None, 7)


In [8]:
history = model.fit(
    [np.array(x_train.to_list())], y_train, batch_size=16, epochs=5, validation_data=([np.array(x_valid.to_list())], y_valid)
)

Epoch 1/5
트랜스포머
트랜스포머
트랜스포머
트랜스포머
1529/1532 [============================>.] - ETA: 0s - loss: 1.9054 - accuracy: 0.2026트랜스포머
트랜스포머
1532/1532 [==============================] - 27s 16ms/step - loss: 1.9052 - accuracy: 0.2028 - val_loss: 1.8794 - val_accuracy: 0.0366
Epoch 2/5
1532/1532 [==============================] - 24s 16ms/step - loss: 1.7999 - accuracy: 0.2753 - val_loss: 1.9024 - val_accuracy: 0.0250
Epoch 3/5
1532/1532 [==============================] - 24s 16ms/step - loss: 1.7305 - accuracy: 0.3082 - val_loss: 1.9610 - val_accuracy: 0.0294
Epoch 4/5
1532/1532 [==============================] - 24s 16ms/step - loss: 1.6816 - accuracy: 0.3280 - val_loss: 1.8370 - val_accuracy: 0.0491
Epoch 5/5
1532/1532 [==============================] - 23s 15ms/step - loss: 1.6437 - accuracy: 0.3411 - val_loss: 1.9119 - val_accuracy: 0.0660


In [9]:
x_test_np = np.array(x_test)

In [10]:
list_col = x_test
list_col_array = np.array(list_col.tolist())
new_df = pd.DataFrame(list_col_array, columns=[f'col{i+1}' for i in range(list_col_array.shape[1])])

In [11]:
new_df

,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,...,col71,col72,col73,col74,col75,col76,col77,col78,col79,col80
0,3.239397,3.234272,3.229146,3.243241,3.245804,3.247085,3.238116,3.240678,3.239397,3.235553,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.143951,4.138826,4.134982,4.131138,4.128576,4.128576,4.129857,4.132420,4.127295,4.119607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.453379,3.458504,3.450816,3.448254,3.443129,3.436722,3.435441,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.079888,4.072201,4.045294,4.041451,4.023513,4.022232,4.014544,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.438003,3.445691,3.480287,3.523852,3.548198,3.563574,3.575106,3.577668,3.582793,3.623796,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2562,1.194537,1.171473,1.150971,1.129189,1.106125,1.093311,1.075373,1.056153,1.043339,1.053590,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2563,0.929301,0.922894,0.917769,0.916488,0.912644,0.903674,0.903674,0.898549,0.892142,0.887017,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2564,0.888298,0.880610,0.878048,0.875485,0.874204,0.867797,0.866516,0.867797,0.858828,0.857546,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2565,0.831920,0.825513,0.821669,0.822950,0.819107,0.816544,0.813981,0.815262,0.806293,0.805012,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
x_test_np = np.array(new_df)
x_test_tensor = tf.convert_to_tensor(x_test_np)
x_test_tensor = tf.cast(x_test_tensor, tf.int64)

In [13]:
y_pred = model.predict(x_test_tensor)

트랜스포머
트랜스포머
81/81 [==============================] - 1s 10ms/step


In [14]:
yp=[]
for i in y_pred:
    yp.append(i.argmax())

## EDA 데이터 (train은 증강한거 test는 증강 안한거 8개 세션)

In [15]:
from sklearn import metrics

# Print the confusion matrix
print(metrics.confusion_matrix(y_test.to_list(), yp))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test.to_list(), yp, output_dict=True, digits=5))

[[  11  196  239   72  236  205 1103]
 [   4   50   30    8   57   28  184]
 [   0    5    3    1    7    4    9]
 [   0    8    8    1    3    3   25]
 [   0    5    4    1    8    3   15]
 [   0    4    3    1    1    4    3]
 [   0    2    2    0    1    2    8]]
{'0': {'precision': 0.7333333333333333, 'recall': 0.00533462657613967, 'f1-score': 0.01059220028887819, 'support': 2062}, '1': {'precision': 0.18518518518518517, 'recall': 0.13850415512465375, 'f1-score': 0.15847860538827258, 'support': 361}, '2': {'precision': 0.010380622837370242, 'recall': 0.10344827586206896, 'f1-score': 0.018867924528301883, 'support': 29}, '3': {'precision': 0.011904761904761904, 'recall': 0.020833333333333332, 'f1-score': 0.015151515151515154, 'support': 48}, '4': {'precision': 0.025559105431309903, 'recall': 0.2222222222222222, 'f1-score': 0.045845272206303724, 'support': 36}, '5': {'precision': 0.01606425702811245, 'recall': 0.25, 'f1-score': 0.03018867924528302, 'support': 16}, '6': {'precision': 

In [18]:
report = metrics.classification_report(y_test.to_list(), yp, output_dict=True, digits=5)
print(report['weighted avg'])

{'precision': 0.615942304085143, 'recall': 0.033112582781456956, 'f1-score': 0.032191661940211405, 'support': 2567}
